In [1]:
import mdtraj as md

In [2]:
from pathlib import Path

In [3]:
!pwd

/home/crystal/Desktop/decaL


In [4]:
traj = md.load_dcd('decaL_prd.dcd', top='decaL_clean.pdb')

In [5]:
traj

<mdtraj.Trajectory with 2000000 frames, 193 atoms, 10 residues, and unitcells at 0x7fb9cbc7bd90>

In [6]:
traj = md.Trajectory.superpose(traj, traj[0], frame=0, atom_indices=None, ref_atom_indices=None, parallel=True)

md.rmsf(traj, traj[0], frame=0, atom_indices=None, parallel=True, precentered=False)

In [7]:
psi_indices = [[  0,   4,   6,  21],
        [ 21,  23,  25,  40],
        [ 40,  42,  44,  59],
        [ 59,  61,  63,  78],
        [ 78,  80,  82,  97],
        [ 97,  99, 101, 116],
        [116, 118, 120, 135],
        [135, 137, 139, 154],
        [154, 156, 158, 173]]
phi_indices = [[  6,  21,  23,  25],
        [ 25,  40,  42,  44],
        [ 44,  59,  61,  63],
        [ 63,  78,  80,  82],
        [ 82,  97,  99, 101],
        [101, 116, 118, 120],
        [120, 135, 137, 139],
        [139, 154, 156, 158],
        [158, 173, 175, 177]]
angles = []
for i in range(min(len(phi_indices), len(psi_indices))):
    phi_set = phi_indices[i]
    psi_set = psi_indices[i]
    angles.append(md.compute_dihedrals(traj, [phi_set, psi_set]))
print(angles)

[array([[-2.3967967 ,  2.1113954 ],
       [-2.5720289 ,  2.4043155 ],
       [-2.6248775 ,  2.146798  ],
       ...,
       [-0.98182493,  2.2614574 ],
       [-1.1026256 ,  2.07684   ],
       [-1.6414802 ,  2.3840368 ]], dtype=float32), array([[-1.4792533,  2.1927392],
       [-2.191297 ,  2.7861776],
       [-2.2276006,  2.5695126],
       ...,
       [-1.4492573,  1.8945514],
       [-1.3929515,  1.6353705],
       [-1.3472149,  2.1931133]], dtype=float32), array([[-2.2232625 , -0.84843457],
       [-1.5233443 , -0.92048067],
       [-2.077363  , -0.5693003 ],
       ...,
       [-1.0680399 , -0.5743143 ],
       [-0.9179571 , -0.43310112],
       [-0.9678982 , -0.68397117]], dtype=float32), array([[-1.6271757 ,  2.1587515 ],
       [-1.910881  ,  2.2375846 ],
       [-1.9017276 ,  2.072853  ],
       ...,
       [-1.2069081 , -0.2982024 ],
       [-0.9997073 , -0.4693609 ],
       [-1.2494581 , -0.17315002]], dtype=float32), array([[-1.6686207 ,  2.2378182 ],
       [-1.3772926 ,

In [9]:
import numpy as np
np.shape(angles)

(9, 2000000, 2)

In [17]:
import pyemma as pm
for i in range(len(angles)):
    phi_values = angles[i][:, 0]
    psi_values = angles[i][:, 1]
    phi_set = phi_indices[i]
    psi_set = psi_indices[i]

In [21]:
def find_transitions(s1_mask, s2_mask, min_len=1):
    s2s1_transition_ids, s1s2_transition_ids = [], []
    
    intermediate_mask = ~np.logical_or(s1_mask, s2_mask)
    intermediate_ids = [i for i, value in enumerate(intermediate_mask) if value]
    split_indices = np.where(np.diff(intermediate_ids) > 1)[0] + 1
    intermediate_chunk_ids = [c for c in np.split(intermediate_ids, split_indices) if len(c)>min_len]

    for c in intermediate_chunk_ids:
        if c[0] == 0 or c[-1] == len(s1_mask)-1: continue 
        if (s1_mask[c[0]-1] == True) and (s2_mask[c[-1]+1] == True):
            s1s2_transition_ids.append([c[0]-1, c[-1]+1])
            with open('s1s2.txt', 'a') as f:
                f.writelines(str(c[0]-1) + ' ' + str(c[-1]+1) + '\n')
        elif (s2_mask[c[0]-1] == True) and (s1_mask[c[-1]+1] == True):
            s2s1_transition_ids.append([c[0]-1, c[-1]+1])
            with open('s2s1.txt', 'a') as f:
                f.writelines(str(c[0]-1) + ' ' + str(c[-1]+1) + '\n')
    return s2s1_transition_ids, s1s2_transition_ids

In [12]:
def save_transitions(traj, s2s1_transition_ids, s1s2_transition_ids):
    if len(s2s1_transition_ids)>0:
        for no, ids in enumerate(s2s1_transition_ids):
            transition_traj = traj[ids[0]:ids[1]+1]
            transition_traj.superpose(transition_traj[0])
            transition_traj.save_pdb(md_transition_path.joinpath(f'traj_s2s1slowest{no}.pdb'))
    if len(s1s2_transition_ids)>0:
        for no, ids in enumerate(s1s2_transition_ids):
            transition_traj = traj[ids[0]:ids[1]+1]
            transition_traj.superpose(transition_traj[0])
            transition_traj.save_pdb(md_transition_path.joinpath(f'traj_s1s2slowest{no}.pdb'))
    return None

In [13]:
from tqdm import tqdm

In [14]:
from pathlib import Path

In [15]:
md_transition_path = Path('./')

In [23]:
import pyemma as pm
for i in range(len(angles)):
    phi_values = angles[i][:, 0]
    psi_values = angles[i][:, 1]
    phi_set = phi_indices[i]
    psi_set = psi_indices[i]
    find_transitions(phi_values < -0.5 , phi_values > 0.5)

In [24]:
s2s1_transition_ids = [
[287522, 287525],
[1089845, 1089850],
[333089, 333092]]
s1s2_transition_ids =  [[231204, 231208],
  [1083454, 1083459]]
save_transitions(traj, s2s1_transition_ids, s1s2_transition_ids)